In [1]:
import pandas as pd
train=pd.read_csv('data/nezha_0.6716_result.csv', sep='\t')

In [6]:
print(train['label'])

0        0.0,0.014421973377466202,0.011997207999229431,...
1        0.0,0.0,0.00037736445665359497,0.0188262239098...
2        0.01639515347778797,0.0009731091558933258,0.0,...
3        0.12240719050168991,0.020964160561561584,0.0,0...
4        0.054671693593263626,0.07297415286302567,0.0,0...
                               ...                        
21371    0.0,0.2460590898990631,0.05887199938297272,0.1...
21372    0.0,0.5472385287284851,0.04858739674091339,0.1...
21373    0.01817462593317032,0.18601173162460327,0.0,0....
21374    0.005028177052736282,0.04220797121524811,0.0,0...
21375    0.00024833157658576965,0.08352718502283096,0.0...
Name: label, Length: 21376, dtype: object


In [7]:
test=pd.read_csv('data/test_dataset.tsv', sep='\t')

In [9]:
content_dict = {}
for index in range(len(test['content'])):
    currentindex = test['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = test['content'][index]

In [10]:
def sortedDictValues(adict): 
    keys = adict.keys() 
    keys = sorted(keys)
    print('keys = ')
    print(keys[0:10])
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict 
new_content_dict = sortedDictValues(content_dict)

keys = 
['01597_0001_A_0001', '01597_0001_A_0002', '01597_0001_A_0003', '01597_0001_A_0004', '01597_0001_A_0005', '01597_0001_A_0006', '01597_0001_A_0007', '01597_0001_A_0008', '01597_0001_A_0009', '01597_0001_A_0010']


In [21]:
train['content'] = test['content']
train['character'] = test['character']

In [17]:
inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

In [18]:
keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    #dict_keys_index[index] = keys[index]
    dict_keys_index[keys[index]] = index

In [22]:
content,emotions = [],[]
dicts = {}
inv_dicts = {}
label_num = 0
characters = []
label1,label2,label3,label4,label5,label6 = [],[],[],[],[],[]
for index in range(len(train['content'])):
    if pd.isna(train['label'][index]) == False:
        content.append(train['content'][index])
        emotions.append(train['label'][index])
        characters.append(train['character'][index])
        current_emotion = train['label'][index].split(',')
        label1.append(float(current_emotion[0]))
        label2.append(float(current_emotion[1]))
        label3.append(float(current_emotion[2]))
        label4.append(float(current_emotion[3]))
        label5.append(float(current_emotion[4]))
        label6.append(float(current_emotion[5]))

In [24]:
import pandas as pd
import itertools
import json
import math
from bertmodels import Config
from nezha import Bert
#prefix = 'chinese-roberta-wwm-ext'

prefix = 'nezha-base'
vocab_file = prefix + r'/vocab.txt'
vocab_size = len(open(vocab_file,'r').readlines()) 
with open(prefix + '/config.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
json_data['vocab_size'] = vocab_size
config = Config(**json_data)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from tokenization import FullTokenizer
import numpy as np
tokenizer = FullTokenizer(vocab_file=vocab_file)
from tqdm import tqdm
config.with_mlm = False
#config.with_pooler = True
bertmodel = Bert(config)
import os
import random

---__init__ Nezha


In [25]:
def seed_torch(seed=1):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(60)

In [26]:
from loader_nezha import load_bert_data
bertmodel = load_bert_data(bertmodel, prefix + '/pytorch_model.bin')

!!!load Pytorch model!!!
Done loading 196 NEZHA weights from: nezha-base/pytorch_model.bin. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	cls.seq_relationship.bias
	cls.predictions.transform.dense.bias
	cls.predictions.transform.LayerNorm.weight
	cls.predictions.transform.dense.weight
	cls.seq_relationship.weight
	cls.predictions.transform.LayerNorm.bias
	cls.predictions.decoder.weight
	bert.pooler.dense.bias
	bert.pooler.dense.weight
	cls.predictions.bias


In [63]:
class ClassificationDataset(Dataset):
    def __init__(self,text,characters,label1,label2,label3,label4,label5,label6,maxlen,flag):
        self.text = text
        self.maxlen = maxlen
        self.characters = characters
        #self.label = [label1,label2,label3,label4,label5,label6]
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_character = characters[index]
            current_token = tokenizer.tokenize(current_text)
            current_str_index = inv_new_content_dict[current_text]

            current_key_index = dict_keys_index[current_str_index]
            while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
                current_key_index = current_key_index-1
            #找寻前面语句的过程之中去重
            if current_key_index == -1:
                pre_content = ''
            else:
                pre_content = new_content_dict[keys[current_key_index]]
            #找到前面去重之后的语句内容
            new_str_index = keys[current_key_index]
            current_str_index = current_str_index.split('_')
            new_str_index = new_str_index.split('_')
            if current_str_index[0] != new_str_index[0]:
                pre_content = ''
            if current_str_index[1] != new_str_index[1]:
                pre_content = ''
            #不属于同一个剧本杀的剧本
            
            
            if str(current_character) == 'nan':
                current_character = '无'
            if pre_content == '':
                pre_content = '无'
            current_character_token = tokenizer.tokenize(current_character)
            pre_token = tokenizer.tokenize(pre_content)
#             current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+current_character_token+["[SEP]"]
#             current_token = pre_token+["[SEP]"]+current_token+["[SEP]"]+current_character_token+["[SEP]"]
            current_token = ["[CLS]"] + pre_token +["[SEP]"] + current_token +["[SEP]"]+ current_character_token +["[SEP]"]

            current_id = tokenizer.convert_tokens_to_ids(current_token)            
            current_id = self.sequence_padding(current_id)
            token_data.append(current_token)
            token_id.append(current_id)
        self.token_data = token_data
        self.token_id = token_id
        self.label = [label1,label2,label3,label4,label5,label6]
        self.tensors = [torch.tensor(self.token_id,dtype=torch.long),
                 torch.tensor(self.label[0],dtype=torch.float),
                 torch.tensor(self.label[1],dtype=torch.float),
                 torch.tensor(self.label[2],dtype=torch.float),
                 torch.tensor(self.label[3],dtype=torch.float),
                 torch.tensor(self.label[4],dtype=torch.float),
                 torch.tensor(self.label[5],dtype=torch.float)]
        
    def __len__(self):
        return len(self.token_id)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)

    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [64]:
text = content
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np
ss = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)
#建立4折交叉验证方法 查一下KFold函数的参数
text = np.array(text)
label1 = np.array(label1)
label2 = np.array(label2)
label3 = np.array(label3)
label4 = np.array(label4)
label5 = np.array(label5)
label6 = np.array(label6)
characters = np.array(characters)
for train_index,test_index in ss.split(text, labelx):
    train_text = text[np.array(train_index)]
    test_text = text[test_index]
    train_characters = characters[train_index]
    test_characters = characters[test_index]
    train_label1 = label1[train_index]
    test_label1 = label1[test_index]
    train_label2 = label2[train_index]
    test_label2 = label2[test_index]
    train_label3 = label3[train_index]
    test_label3 = label3[test_index]
    train_label4 = label4[train_index]
    test_label4 = label4[test_index]
    train_label5 = label5[train_index]
    test_label5 = label5[test_index]
    train_label6 = label6[train_index]
    test_label6 = label6[test_index]

In [65]:
labelx = np.rint(label5,)
print(labelx)

[0. 0. 0. ... 0. 0. 0.]


In [66]:
dict1 = {}
for item in labelx:
    if item not in dict1.keys():
        dict1[item] = 1
    else:
        dict1[item] += 1
dict1

{0.0: 19941, 1.0: 1224, 2.0: 211}

In [67]:
train_dataset = ClassificationDataset(train_text,train_characters,train_label1,train_label2,train_label3,train_label4,train_label5,train_label6,maxlen=200,flag=True)
test_dataset = ClassificationDataset(test_text,test_characters,test_label1,test_label2,test_label3,test_label4,test_label5,test_label6,maxlen=200,flag=False)
#到里面的classificationdataset才进行字符的切割以及划分
train_loader = DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=10)

100%|██████████| 4275/4275 [00:01<00:00, 3169.27it/s]


In [68]:
class ClassificationModel(nn.Module):
    def __init__(self,model,config,n_labels):
        super(ClassificationModel,self).__init__()
        #self.embedding = nn.Embedding(30522,768)
        self.model = model
        self.fc1 = nn.Linear(config.embedding_size,128)
        self.activation = F.relu
        self.dropout = nn.Dropout(0.2)
        #self.activation = F.tanh
        self.fc2 = nn.Linear(128, n_labels)

        
    def forward(self,input_ids,segment_ids,input_mask):
        #outputs = self.embedding(input_ids)
        output = self.model(input_ids)
        #[64,128,768]
        output = self.dropout(output)
        output = output[:,0]
        output = self.fc1(output)
        output = self.activation(output)
        pooled_output = self.dropout(output)
        logits = self.fc2(pooled_output)
        
        return logits
    #之前这里少量return outputs返回值为None

In [69]:
def compute_multilabel_loss(x,model,label):
    logit = model(x,None,None)
    mseloss = 0
    loss_fn = torch.nn.MSELoss(reduce=True, size_average=True)
    
    logit = torch.transpose(logit, 0, 1)
    mseloss = loss_fn(logit,label)
    return mseloss

In [70]:
from tqdm import tqdm
n_label = 6
model = ClassificationModel(bertmodel,config,n_label)

In [82]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=9e-6)
#不动态调整学习率的时候，lr=2e-5的时候最好值0.420293
def lr_lambda(epoch):
    if epoch > 5:
        return 1
    else:
        return 2/(epoch+1)
scheduler = LambdaLR(optimizer, lr_lambda)
print("初始化的学习率：", optimizer.defaults['lr'])
bestpoint = 0.0
for epoch in range(5):
    print('epoch {}'.format(epoch))
    train_loss = 0
    train_acc = 0
    
    model.train()
    
    model = model.to(device)
    model = nn.DataParallel(model)
    loss_fn = torch.nn.MSELoss(reduce=True,size_average=True)
    
    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(train_loader):
        torch.set_printoptions(edgeitems=768)# 设置输出矩阵维度为768
        #print('batch_token_ids')
        #print(batch_token_ids)
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = [batch_label1.numpy(),batch_label2.numpy(),batch_label3.numpy(),\
                        batch_label4.numpy(),batch_label5.numpy(),batch_label6.numpy()]
        batch_labels = torch.tensor(batch_labels,dtype=torch.float)
        batch_labels = batch_labels.to(device)
        #for index in range(len(batch_labels)):
        #    batch_labels[index] = batch_labels[index].to(device)
        optimizer.zero_grad()
        loss = compute_multilabel_loss(batch_token_ids,model,batch_labels)
        train_loss = train_loss+loss
        loss.backward()
        optimizer.step()
    scheduler.step()
    print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
    #注意必须从epoch=1开始，否则第0个没有学习率
    print('Train Loss: {:.6f}'.format(train_loss))
    
    model = model.to(device)
    model.eval()
    
    eval_true_label = [[],[],[],[],[],[]]
    eval_predict_label = [[],[],[],[],[],[]]
    
    eval_label_loss = [[0,0,0,0],\
                       [0,0,0,0],\
                       [0,0,0,0],\
                       [0,0,0,0]]
    #for batch_token_ids,batch_labels in tqdm(test_loader,bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET)):
    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(test_loader):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = [batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6]
        with torch.no_grad():
            output = model(batch_token_ids,None,None)
        for index in range(len(output)):
            #总的数据
            for index1 in range(len(output[index])):
                #对应的类别概率0~6
                abs0 = abs(output[index][index1] - 0)
                abs1 = abs(output[index][index1] - 1)
                abs2 = abs(output[index][index1] - 2)
                abs3 = abs(output[index][index1] - 3)
                currentdata = [abs0,abs1,abs2,abs3]
                current_label = currentdata.index(min(currentdata))
                #eval_predict_label[index1].append(current_label)
                current_predict = output[index][index1].item()
                if current_predict < 0.01:
                    current_predict = 0
                elif current_predict > 3:
                    current_predict = 3
                #当前类别的分类结果,这里append(output[index][index1])
                #直接放入对应概率值时效果最好,这里直接放入0,1,2,3对应的数值
                eval_predict_label[index1].append(current_predict)
        for index in range(len(batch_labels)):
            current_label = np.array(batch_labels[index].cpu()).tolist()
            eval_true_label[index].extend(current_label)
    criterion = nn.MSELoss()
    totalloss = 0

    for index in range(len(eval_true_label)):
        inputs = torch.autograd.Variable(torch.from_numpy(np.array(eval_predict_label[index])))
        target = torch.autograd.Variable(torch.from_numpy(np.array(eval_true_label[index])))
        for index1 in range(len(inputs)):
            abs0 = abs(inputs[index1]-0)
            abs1 = abs(inputs[index1]-1)
            abs2 = abs(inputs[index1]-2)
            abs3 = abs(inputs[index1]-3)
            currentdata = [abs0,abs1,abs2,abs3]
            current_label = currentdata.index(min(currentdata))
            true_label = int(np.rint(target[index1].item()))
            if current_label != true_label:
                eval_label_loss[true_label][current_label] = eval_label_loss[true_label][current_label]+1
                #对的预测为错误的
        current_loss = criterion(inputs.float(),target.float())
        current_loss = current_loss.item()
        print('index = %d,current_loss = %f'%(index,current_loss))
        totalloss = totalloss+current_loss
    
    #totalloss = totalloss/len(eval_predict_label)
    print('totalloss = ')
    print(totalloss)
    totalloss = totalloss/6
    totalloss = math.sqrt(totalloss)
    currentpoint = 1/(1+totalloss)
    #currentpoint = 1/(1+current_loss)
    print('current_point = ')
    print(currentpoint)
    if currentpoint > bestpoint:
        bestpoint = currentpoint
        torch.save(model,'best_score'+str(bestpoint)+'.pth')
    print('eval_label_loss = ')
    print(eval_label_loss)

初始化的学习率： 9e-06
epoch 0


100%|██████████| 1711/1711 [03:55<00:00,  7.26it/s]


第0个epoch的学习率：0.000009
Train Loss: 17.115278


100%|██████████| 428/428 [00:24<00:00, 17.47it/s]


index = 0,current_loss = 0.006356
index = 1,current_loss = 0.016710
index = 2,current_loss = 0.007239
index = 3,current_loss = 0.024105
index = 4,current_loss = 0.015312
index = 5,current_loss = 0.029234
totalloss = 
0.09895640518516302
current_point = 
0.8861916833061471
eval_label_loss = 
[[0, 567, 3, 0], [393, 0, 44, 0], [5, 125, 0, 0], [0, 0, 0, 0]]
epoch 1


...

index = 0,current_loss = 0.005667
index = 1,current_loss = 0.008623
index = 2,current_loss = 0.004836
index = 3,current_loss = 0.019212
index = 4,current_loss = 0.012754
index = 5,current_loss = 0.022043
totalloss = 
0.07313401577994227
current_point = 
0.9005732738099107
eval_label_loss = 
[[0, 474, 2, 0], [341, 0, 43, 0], [0, 94, 0, 0], [0, 0, 0, 0]]
epoch 4


100%|██████████| 1711/1711 [04:02<00:00,  7.06it/s]


第4个epoch的学习率：0.000003
Train Loss: 8.415354


100%|██████████| 428/428 [00:25<00:00, 16.56it/s]


index = 0,current_loss = 0.004932
index = 1,current_loss = 0.008749
index = 2,current_loss = 0.005315
index = 3,current_loss = 0.018521
index = 4,current_loss = 0.013161
index = 5,current_loss = 0.022793
totalloss = 
0.07347233686596155
current_point = 
0.9003664499230284
eval_label_loss = 
[[0, 502, 2, 0], [337, 0, 46, 0], [0, 97, 0, 0], [0, 0, 0, 0]]


In [72]:
for index in range(len(output)):
    print(output[index])
    break

tensor([0.0150, 0.0570, 0.0097, 0.0733, 0.0384, 0.0724], device='cuda:0')


AttributeError: 'numpy.float64' object has no attribute 'type'

In [80]:
type(int(np.rint(0.7)))

int